# Day 3: データの結合やデータ変換のためのSQL操作
Day 2に引き続き，Day 3では関係データベース上に格納された購買データを用いた演習を行う．
Day 3では，
* 複数の表データの結合
* 関係データベース上での数値計算
* 分析のためのデータの変換

のためのSQL操作について体験する．

なお，Day 2からDay 5の演習で用いるデータセットは，[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)で提供されているデータセットをSQLite形式に変換したものである．また，「ノック」は[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)に収録されている演習課題を，本授業向けにアレンジしたものである．「課題」は完全に山本が作成したものである．

---

## 今週のSQLを用いた課題
前回と同様にDeepnote上でBigQueryと接続して

なお，Deepnote上でコードを実行するには，以下の2つの方法がある：
* 「再生ボタン」をクリックする
* コードが書かれたセルをクリックし，Shift + Enterキーを押す

---

## Deepnote上でのSQLの実行方法

セルを「+」で追加する際に、セルの型をBigQuery型に指定すると、そのセル内に直接SQLを書くことができる。

前回と同様に、テーブル名の前には毎回データベース名を明示的に指定する必要がある点に注意する。

In [ ]:
df_1 = _deepnote_execute_sql('SELECT\n    * \nFROM \n    virtual_store.INFORMATION_SCHEMA.TABLES', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,nagoya-city-university,virtual_store,category,BASE TABLE,YES,NO,2023-06-11 06:36:09.802000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None
1,nagoya-city-university,virtual_store,geocode,BASE TABLE,YES,NO,2023-06-11 06:41:04.340000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None
2,nagoya-city-university,virtual_store,product,BASE TABLE,YES,NO,2023-06-11 06:43:28.871000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None
3,nagoya-city-university,virtual_store,receipt,BASE TABLE,YES,NO,2023-06-11 06:44:47.969000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None
4,nagoya-city-university,virtual_store,store,BASE TABLE,YES,NO,2023-06-11 06:51:05.482000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None
5,nagoya-city-university,virtual_store,customer,BASE TABLE,YES,NO,2023-06-11 06:38:11.656000+00:00,None,None,None,None,CREATE TABLE `nagoya-city-university.virtual_s...,NULL,None


上記SQLは，データベースに格納されたテーブルの一覧を表示するものである．
今回使用するデータベース，つまり今回扱う杏森堂のデータベースには，**6つ**のテーブルが格納されていることが分かる．なお，対象データベースの構造が記された実体関連図（ER図）は[コチラ](https://github.com/hontolab-courses/data-analytics-2022/raw/main/doc/ER-diagram.pdf)から確認することできる．

課題とドリルに取り組む前に，あと1つ，別のSQLを実行してみよう．
以下のSQLを実行すると，`receipt`テーブルの最初の10件を表示することができる
（SQL文に`LIMIT`を付けないと，テーブル内の全データが表示されてしまうので注意）．

（ちなみに、「テーブル一覧を表示する」の書き方は、使うデータベース管理ソフトの種類によって異なる）

In [ ]:
df_2 = _deepnote_execute_sql('SELECT * FROM virtual_store.receipt LIMIT 10; ', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_2

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20190308,1552003200,S13001,112,1,CS001113000004,P071001012,1,98
1,20190308,1552003200,S13001,112,2,CS001113000004,P071401009,1,1200
2,20180503,1525305600,S13001,112,2,CS001114000005,P080803001,1,100
3,20180503,1525305600,S13001,112,1,CS001114000005,P070101119,1,338
4,20190731,1564531200,S13001,112,2,CS001114000005,P050101001,1,40
5,20190731,1564531200,S13001,112,1,CS001114000005,P070301002,1,148
6,20171228,1514419200,S13001,112,1,CS001115000010,P071401019,1,2200
7,20171228,1514419200,S13001,112,2,CS001115000010,P071001073,1,120
8,20180701,1530403200,S13001,112,2,CS001115000010,P070802002,1,98
9,20180701,1530403200,S13001,112,1,CS001115000010,P070101003,1,48


---

## 課題2
課題2-1，課題2-2のいずれかを選択し，結果を得るためのSQL文を書き，実行結果とともに示しなさい．

### 課題2-1: 月別売上の対昨年比

レシート明細テーブル（`receipt`）には2017年1月から2019年10月の間の購買情報が記録されている．2017年から2019年までの期間の売上を把握するために，1ヶ月ごとに以下の情報を集約表示せよ：
* 年月（`year_month`）
* 購買回数（`purchase_freq`）
* 月間売上高（`total_amount`）
* 購買1回あたりの平均購入額（`avg_amount_per_purchase`）
* 当該月の前年売上高（`total_amount_last_year`）
* 売上の対前年比（`ratio`）

※ 以下は，SQL実行結果のイメージである：

| year_month | purchase_freq | total_amount | avg_amount_per_purchase | total_amount_last_year | ratio |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 2017-01 | 1405 | 902056 | 642 | NULL | NULL  |
| 2017-02 | 1219 | 764413 | 627 | NULL | NULL  |
| ... | ... | ... | ... | ... | ... |
| 2017-12 | 1467 | 939654 | 640 | NULL | NULL  |
| 2018-01 | 1473 | 944509 | 641 | 902056 | 1.05 |
| 2018-02 | 1387 | 864128 | 623 | 764413 | 1.13 |
| ... | ... | ... | ... | ... | ... |
| 2019-10 | 1750 | 1143062 | 653 | 1069939 | 1.07 |

In [ ]:
df_3 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_3

### 課題2-2: ABC分析

ABC分析は販売戦略を考えるために，売上によって商品をランク付けする手法である．
一般に，ABC分析では売上総額の
* 上位0〜70％の商品をAランク
* 上位70〜90％の商品をBランク
* 上位90〜100%の商品をCランク

とするランク付けを行う．

商品カテゴリ「菓子（`category_major_cd = 08`）」に属する商品について，小区分（`category_small_name`）ごとに売上を集計し，菓子カテゴリの売上総額に占める割合（構成比）を計算せよ．
また，売上がN位の小区分の行には売上額上位N位までの構成比累計，および構成比累計に基づくABC分析のランク付け結果も表示せよ．

※ 以下は，SQL実行結果のイメージである：

| カテゴリ名 | 売上 | 構成比 | 累積構成比 | ランク |
| ---- | ---- | ---- | ---- | ---- |
| 煎餅 | 194211 | 0.1306 | 0.1306 | A |
| 菓子詰め合わせ | 129580 | 0.0871 | 0.2178 | A |
| ナッツ類 | 129020 | 0.0867 | 0.3046 | A |
| ... | ... | ... | ... | ... |
| キャンディー | 46177 | 0.0310 | 0.7208 | B |
| ... | ... | ... | ... | ... |
| その他珍味 | 24086 | 0.0162 | 0.9073 | C |
| ... | ... | ... | ... | ... |

In [ ]:
df_4 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_4

---
## ノック

### Knock 41: 結合1
レシート明細テーブル（`receipt`）と店舗テーブル（`store`）を内部結合し，レシート明細テーブルの全項目と店舗テーブルの店舗名（`store_name`）を10件表示させよ．

In [ ]:
df_5 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_5

### Knock 42: 結合2
商品テーブル（`product`）とカテゴリテーブル（`category`）を内部結合し，商品テーブルの全項目とカテゴリテーブルの小区分名（`category_small_name`）を10件表示させよ．

In [ ]:
df_6 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_6

### Knock 43: 結合3
顧客テーブル（`customer`）とレシート明細テーブル（`receipt`）から，各顧客ごとの売上金額合計を求めよ．ただし，売上実績がない顧客については売上金額を0として表示させること．また，顧客は性別コード（`gender_cd`）が女性（1）であるものを対象とし，非会員（顧客IDが"Z"から始まるもの）は除外すること．なお，表示項目は顧客ID（`customer_id`），顧客名（`customer_name`），売上金額合計とせよ．結果は10件だけ表示させればよい．

※ヒント：`IFNULL（変数1, 0）`関数は，変数1が空値（None）のとき0に置き換える

In [ ]:
df_7 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_7

### Knock 44: 結合4

店舗と商品を組み合わせが全部で何通り「あり得るか」を計算したい．店舗（`store`）と商品（`product`）を直積を求めることで組合せ数を計算せよ．

In [ ]:
df_8 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_8

### Knock 45: 結合5（難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，その結果を以下の3つを項目（列）として持つ表を出力せよ．
* 日付（`sales_ymd`）
* その日の売上金額（`total_amount`）
* その日の1週間前の売上金額（`one_week_before`）

なお，計算結果は10件表示すればよい．このノックはウィンドウ関数の`LAG`関数を使わずに取り組むこと．

* ヒント1：WITH句を使って集計結果のテーブルを一時的に保存する
* ヒント2：Window関数のROW_NUMBER関数を使う

In [ ]:
df_9 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_9

### Knock 46: 結合6

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，前日からの売上金額増減を計算せよ．前日の売上金額の取得にはWindow関数の`LAG`関数を用いること．計算結果は10件表示すればよい．

In [ ]:
df_10 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_10

### Knock 47: 結合7（難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を日付（`sales_ymd`）ごとに集計し，前日からの売上金額増減を計算せよ．なお，計算結果は10件表示すればよい．このノックはウィンドウ関数の`LAG`関数を使わずに取り組むこと．

※ヒント1：WITH句を使って集計結果のテーブルを一時的に保存する
※ヒント2：Window関数のROW_NUMBER関数を使う

In [ ]:
df_11 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_11

### Knock 48: 欠損値1

商品テーブル（`product`）には単価（`unit_price`）と原価（`unit_cost`）が欠損しているレコードが存在する．そのようなレコードを表示せよ．

In [ ]:
df_12 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_12

### Knock 51: 四則演算1

商品テーブル（`product`）の単価（`unit_price`）と原価（`unit_cost`）から，各商品の利益（`profit`）および利益率（`profit_rate`）を算出せよ．
ただし，単価と原価にはNULL値が存在することに注意せよ．なお結果は10件表示させればよい．

In [ ]:
df_15 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_15

### Knock 52: 四則演算2

商品テーブル（`product`）の各商品について，利益率が30%となる新たな単価（`new_unit_price`）を求めよ．その際，1円未満を丸めること（四捨五入または偶数への丸めでよい）．ただし，単価と原価にはNULLが存在することに注意せよ．なお，結果を10件表示すればよい．

In [ ]:
df_16 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_16

### Knock 53: 四則演算と集約演算1

レシート明細テーブル（`receipt`）と顧客テーブル（`customer`）を結合し，性別（`gender`）と年代（`age`から計算）ごとの売上金額（`amount`）の合計値を表示せよ．ただし，表示項目の構成は性別，年代，売上総額の3項目とすること．また，年代は10歳ごとの階級とすること．

In [ ]:
df_17 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_17

### Knock 54: 四則演算と集約演算2（やや難）

レシート明細テーブル（`receipt`）と商品テーブル（`product`）を結合し，顧客毎に全商品の売上金額合計とカテゴリ大区分（`category_major_cd`）が`"07"`（瓶詰缶詰）の売上金額合計を計算の上，両者の比率を求めよ．
抽出対象はカテゴリ大区分`"07"`（瓶詰缶詰）の売上実績がある顧客のみとし，結果は10件表示させればよい．

※ヒント: Window関数の`OVER`句の中で`PARTION BY`を使う

In [ ]:
df_18 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_18

### Knock 55: 四則演算と集約演算3（やや難）

レシート明細テーブル（`receipt`）を用いて顧客ID（`customer_id`）ごとの総売上を計算し，その上位10件を表示せよ．
その際，売上順位がN位の顧客IDの行には売上額上位N位までの顧客の累積売上額を併せて表示すること．
なお，顧客IDが"Z"から始まるのものは非会員を表すため，除外して計算すること．

※ヒント: Window関数の`OVER`句の中で`ORDER BY`を使うと…

In [ ]:
df_19 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_19

### Knock 56: 2値化

顧客テーブル（`customer`）を東京（先頭3桁が100〜209のもの）とそれ以外の地域に分け，レシート明細テーブル（`receipt`）と結合し，全期間における購買頻度（`receipt_no`の総数）および売上総額を集計せよ．

※ヒント: `CASE`句を使う

In [ ]:
df_20 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_20

### Knock 57: カテゴリ化1

顧客テーブル（`customer`）の年齢（`age`）をもとに10歳刻みで年代を算出し，年代ごとの顧客人数を集計せよ．なお，年代は10代であれば`1`，20代であれば`2`のようにすればよい．年代を表すカテゴリ名は任意とする．

※ 小数を整数に変換する必要があれば`CAST`関数を使うこと

In [ ]:
df_21 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_21

### Knock 58: カテゴリ化2（やや難）

レシート明細テーブル（`receipt`）の売上金額（`amount`）を顧客ID（`customer_id`）ごとに合計し，顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し，顧客ID、売上金額合計とともに表示せよ．なお，表示件数は10件でよい．
カテゴリ値は以下のようにすること．
- 最小値以上第1四分位未満: 1を付与
- 第1四分位以上第2四分位未満: 2を付与
- 第2四分位以上第3四分位未満: 3を付与
- 第3四分位以上: 4を付与

※ ヒント: `PERCENT_RANK`関数を使う

In [ ]:
df_22 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_22

### Knock 59: 日付型1

レシート明細テーブル（`receipt`）の売上日（`sales_epoch`）は購買行動が行われた日付が[UNIX時刻](https://ja.wikipedia.org/wiki/UNIX時間)で格納されている．`sales_epoch`の数値を`DATETIME`関数によって日付時刻オブジェクトに変換し，2018年のレシート明細のみを表示せよ．表示件数は10件でよい．

日時の取り扱い方法は、使用しているデータベース管理ソフトによって異なる。
たとえばSQLiteデータベースではstrftime関数を使うことで，日付時刻オブジェクトから年や月，日を取り出すことができる．MySQLデータベース等ではDATE_FORMAT関数を使うことで同様の操作ができる．

今回の授業では、Google BigQueryを用いるため、以下のページなどを参考に、日時情報を扱う。
https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions

In [ ]:
df_23 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_23

### Knock 60: 日付型2

レシート明細テーブル（`receipt`）に格納された売上金額（`amount`）を年月ごとに集約して表示せよ．すなわち，ある年のある月の売上総額が分かるように結果を表示せよ．なお，表示件数は先頭の10件でよい．

日時の取り扱い方法は、使用しているデータベース管理ソフトによって異なる。
たとえばSQLiteデータベースでは`strftime`関数を使うことで，日付時刻オブジェクトから年や月，日を取り出すことができる．MySQLデータベース等では`DATE_FORMAT`関数を使うことで同様の操作ができる．

今回の授業では、Google BigQueryを用いるため、以下のページなどを参考に、日時情報を扱う。
https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions

In [ ]:
df_24 = _deepnote_execute_sql('', 'SQL_15E9A3D6_608A_4814_B06A_3A0582ED6165', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_24

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8c0a6d85-5d65-49d1-a563-61fa4ced03bc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>